# 臨床試驗文件自動化系統
# Clinical Trial Document Automation System

---

## 📋 系統簡介

本系統整合了五個核心模組，提供完整的臨床試驗文件自動化解決方案：

### 🔧 核心模組

1. **Protocol Parser（試驗書解析器）**
   - 從 Protocol PDF 自動提取關鍵資訊
   - 使用 Google Gemini AI 進行智能解析
   - 支援中英文 Protocol

2. **CRF Generator（病例報告表生成器）**
   - 根據 Protocol 資訊自動生成 CRF 表單
   - 支援多種欄位類型和驗證規則
   - 輸出專業格式的 Word 文件

3. **DVP Generator（資料驗證計畫生成器）**
   - 自動生成資料驗證規則
   - 包含範圍檢查、一致性檢查、Protocol 偏差檢查
   - 輸出詳細的 DVP 文件

4. **User Guide Generator（使用者手冊生成器）**
   - 自動生成 CRF 填寫說明文件
   - 包含詳細的欄位說明和填寫範例
   - 支援螢幕截圖佔位符

5. **Word Formatter（文件格式化工具）**
   - 統一文件格式和樣式
   - 添加頁首、頁尾、Logo
   - 生成專業的臨床試驗文件

### ⚡ 主要功能

- ✅ **一鍵生成**: 上傳 Protocol PDF，自動生成所有文件
- ✅ **互動式介面**: 使用 ipywidgets 提供友善的操作介面
- ✅ **進度追蹤**: 即時顯示生成進度
- ✅ **品質保證**: 自動檢查資料完整性
- ✅ **彈性設定**: 支援自訂參數和樣式
- ✅ **批次下載**: 一次下載所有生成的文件

### 📦 生成文件

1. **CRF (Case Report Form)** - 病例報告表
2. **DVP (Data Validation Plan)** - 資料驗證計畫
3. **User Guide** - CRF 使用者手冊
4. **Protocol Info JSON** - Protocol 資訊結構化資料

---

## 🚀 快速開始

### 使用步驟：

1. **執行 Section 1-2**: 安裝依賴和設定環境
2. **執行 Section 3**: 上傳 Protocol PDF 和設定參數
3. **執行 Section 8**: 一鍵生成所有文件
4. **執行 Section 9**: 下載生成的文件

### 或者逐步執行：

- Section 4: Protocol 資訊提取
- Section 5: 生成 CRF
- Section 6: 生成 DVP
- Section 7: 生成 User Guide

---

**作者**: Clinical Document Automation Team  
**版本**: 1.0.0  
**更新日期**: 2025-11-18

---

# Section 1: 系統安裝

## 📥 安裝所需套件

這個步驟會安裝所有必要的 Python 套件。

In [ ]:
%%capture
# 安裝必要的套件（靜默模式）
!pip install -q pdfplumber google-generativeai python-docx Pillow lxml python-dotenv pandas openpyxl ipywidgets

# 顯示安裝完成訊息
print("✓ 所有套件已成功安裝")

In [ ]:
# 驗證套件安裝
import sys

required_packages = [
    ('pdfplumber', 'PDF 處理'),
    ('google.generativeai', 'Google Gemini AI'),
    ('docx', 'Word 文件處理'),
    ('PIL', '圖片處理'),
    ('ipywidgets', '互動式介面'),
    ('pandas', '資料處理')
]

print("套件安裝驗證：")
print("=" * 60)

all_installed = True
for package, description in required_packages:
    try:
        __import__(package)
        print(f"✓ {package:30} - {description}")
    except ImportError:
        print(f"✗ {package:30} - {description} (未安裝)")
        all_installed = False

print("=" * 60)
if all_installed:
    print("\n✓ 所有套件已正確安裝，可以繼續！")
else:
    print("\n⚠ 部分套件未安裝，請重新執行安裝步驟")

---

# Section 2: 環境設定

## 🔧 導入模組和初始化設定

In [ ]:
# 基礎套件
import os
import sys
import json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 資料處理
import pandas as pd

# 互動式介面
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# 檢測運行環境
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import files
    print("✓ 運行環境: Google Colab")
else:
    print("✓ 運行環境: Jupyter Notebook")

print("✓ 基礎模組導入成功")

In [ ]:
# 設置專案結構
# 如果在 Colab 中，需要先上傳或克隆專案檔案

if IN_COLAB:
    # 選項 1: 從 GitHub 克隆（如果專案在 GitHub 上）
    # !git clone https://github.com/your-repo/clinical-doc-automation.git
    # sys.path.insert(0, '/content/clinical-doc-automation')
    
    # 選項 2: 手動上傳模組檔案
    print("請確保已上傳 'modules' 資料夾到 Colab")
    print("如果尚未上傳，請使用左側檔案面板上傳整個 modules 資料夾")
    
    # 設定模組路徑
    if not os.path.exists('modules'):
        print("\n⚠️ 警告: modules 資料夾不存在")
        print("請上傳 modules 資料夾後重新執行此 cell")
    else:
        sys.path.insert(0, os.getcwd())
        print("✓ 模組路徑已設定")
else:
    # 本地環境，假設在專案根目錄
    project_root = Path.cwd().parent if Path.cwd().name == 'clinical-doc-automation' else Path.cwd()
    sys.path.insert(0, str(project_root))
    print(f"✓ 專案根目錄: {project_root}")

# 建立輸出資料夾
output_dir = Path('output')
output_dir.mkdir(exist_ok=True)
print(f"✓ 輸出資料夾: {output_dir.absolute()}")

In [ ]:
# 導入自訂模組
try:
    from modules.protocol_parser import ProtocolParser, ProtocolInfo as ProtocolInfoParser
    print("✓ Protocol Parser 導入成功")
    PROTOCOL_PARSER_AVAILABLE = True
except ImportError as e:
    print(f"✗ Protocol Parser 導入失敗: {e}")
    PROTOCOL_PARSER_AVAILABLE = False

try:
    from modules.crf_generator import CRFGenerator
    print("✓ CRF Generator 導入成功")
    CRF_GENERATOR_AVAILABLE = True
except ImportError as e:
    print(f"✗ CRF Generator 導入失敗: {e}")
    CRF_GENERATOR_AVAILABLE = False

try:
    from modules.dvp_generator import (
        DVPGenerator, 
        ProtocolInfo, 
        CRFField,
        Severity,
        ValidationType
    )
    print("✓ DVP Generator 導入成功")
    DVP_GENERATOR_AVAILABLE = True
except ImportError as e:
    print(f"✗ DVP Generator 導入失敗: {e}")
    DVP_GENERATOR_AVAILABLE = False

try:
    from modules.user_guide_generator import UserGuideGenerator, ScreenshotPlaceholder
    print("✓ User Guide Generator 導入成功")
    USER_GUIDE_AVAILABLE = True
except ImportError as e:
    print(f"✗ User Guide Generator 導入失敗: {e}")
    USER_GUIDE_AVAILABLE = False

try:
    from modules.word_formatter import WordFormatter
    print("✓ Word Formatter 導入成功")
    WORD_FORMATTER_AVAILABLE = True
except ImportError as e:
    print(f"✗ Word Formatter 導入失敗: {e}")
    WORD_FORMATTER_AVAILABLE = False

# 檢查所有模組狀態
print("\n" + "=" * 60)
all_modules = [
    PROTOCOL_PARSER_AVAILABLE,
    CRF_GENERATOR_AVAILABLE,
    DVP_GENERATOR_AVAILABLE,
    USER_GUIDE_AVAILABLE,
    WORD_FORMATTER_AVAILABLE
]

if all(all_modules):
    print("✓ 所有核心模組已成功導入！")
else:
    print("⚠️ 部分模組導入失敗，請檢查 modules 資料夾是否完整")

---

# Section 3: 使用者介面設定

## 📤 檔案上傳和參數設定

In [ ]:
# API Key 設定
print("🔑 Google Gemini API Key 設定")
print("=" * 60)
print("\n如何獲取免費的 Gemini API Key:")
print("1. 訪問 https://makersuite.google.com/app/apikey")
print("2. 使用 Google 帳號登入")
print("3. 點擊 'Create API Key'")
print("4. 複製 API 金鑰並貼到下方\n")

# API Key 輸入框
api_key_input = widgets.Password(
    placeholder='請輸入您的 Gemini API Key',
    description='API Key:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

api_status = widgets.HTML(value="")

def on_api_key_change(change):
    if change['new']:
        os.environ['GEMINI_API_KEY'] = change['new']
        api_status.value = "<span style='color: green;'>✓ API Key 已設定</span>"
    else:
        api_status.value = "<span style='color: orange;'>⚠ 請輸入 API Key</span>"

api_key_input.observe(on_api_key_change, names='value')

display(api_key_input, api_status)

In [ ]:
# 檔案上傳介面
print("📤 檔案上傳")
print("=" * 60)

# 建立上傳按鈕
upload_protocol_btn = widgets.Button(
    description='📄 上傳 Protocol PDF',
    button_style='primary',
    layout=widgets.Layout(width='250px', height='40px')
)

upload_logo_btn = widgets.Button(
    description='🖼️ 上傳 Logo 圖片 (選填)',
    button_style='info',
    layout=widgets.Layout(width='250px', height='40px')
)

upload_template_btn = widgets.Button(
    description='📋 上傳 Word 範本 (選填)',
    button_style='info',
    layout=widgets.Layout(width='250px', height='40px')
)

# 狀態顯示
protocol_status = widgets.HTML(value="<span style='color: gray;'>未上傳</span>")
logo_status = widgets.HTML(value="<span style='color: gray;'>未上傳</span>")
template_status = widgets.HTML(value="<span style='color: gray;'>未上傳</span>")

# 全域變數儲存檔案路徑
uploaded_files = {
    'protocol_pdf': None,
    'logo_image': None,
    'word_template': None
}

def upload_protocol(b):
    if IN_COLAB:
        uploaded = files.upload()
        if uploaded:
            filename = list(uploaded.keys())[0]
            if filename.lower().endswith('.pdf'):
                uploaded_files['protocol_pdf'] = filename
                protocol_status.value = f"<span style='color: green;'>✓ {filename}</span>"
            else:
                protocol_status.value = "<span style='color: red;'>✗ 請上傳 PDF 檔案</span>"
    else:
        protocol_status.value = "<span style='color: orange;'>⚠ 請手動設定 PDF 路徑</span>"

def upload_logo(b):
    if IN_COLAB:
        uploaded = files.upload()
        if uploaded:
            filename = list(uploaded.keys())[0]
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                uploaded_files['logo_image'] = filename
                logo_status.value = f"<span style='color: green;'>✓ {filename}</span>"
            else:
                logo_status.value = "<span style='color: red;'>✗ 請上傳圖片檔案</span>"
    else:
        logo_status.value = "<span style='color: orange;'>⚠ 請手動設定圖片路徑</span>"

def upload_template(b):
    if IN_COLAB:
        uploaded = files.upload()
        if uploaded:
            filename = list(uploaded.keys())[0]
            if filename.lower().endswith('.docx'):
                uploaded_files['word_template'] = filename
                template_status.value = f"<span style='color: green;'>✓ {filename}</span>"
            else:
                template_status.value = "<span style='color: red;'>✗ 請上傳 Word 檔案</span>"
    else:
        template_status.value = "<span style='color: orange;'>⚠ 請手動設定 Word 路徑</span>"

upload_protocol_btn.on_click(upload_protocol)
upload_logo_btn.on_click(upload_logo)
upload_template_btn.on_click(upload_template)

# 顯示上傳介面
display(widgets.HBox([upload_protocol_btn, protocol_status]))
display(widgets.HBox([upload_logo_btn, logo_status]))
display(widgets.HBox([upload_template_btn, template_status]))

In [ ]:
# 參數設定介面
print("\n⚙️ 生成參數設定")
print("=" * 60)

# 試驗基本資訊
sponsor_input = widgets.Text(
    placeholder='例如: ABC 製藥公司',
    description='贊助商:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

phase_dropdown = widgets.Dropdown(
    options=['Phase I', 'Phase II', 'Phase III', 'Phase IV', 'Pilot Study'],
    value='Phase III',
    description='試驗階段:',
    style={'description_width': 'initial'}
)

# 文件生成選項
generate_crf = widgets.Checkbox(
    value=True,
    description='生成 CRF (病例報告表)',
    style={'description_width': 'initial'}
)

generate_dvp = widgets.Checkbox(
    value=True,
    description='生成 DVP (資料驗證計畫)',
    style={'description_width': 'initial'}
)

generate_guide = widgets.Checkbox(
    value=True,
    description='生成 User Guide (使用者手冊)',
    style={'description_width': 'initial'}
)

# AI 模型選擇
model_dropdown = widgets.Dropdown(
    options=['gemini-1.5-flash', 'gemini-1.5-pro'],
    value='gemini-1.5-flash',
    description='AI 模型:',
    style={'description_width': 'initial'}
)

# 顯示參數設定
display(HTML("<h4>試驗基本資訊</h4>"))
display(sponsor_input)
display(phase_dropdown)

display(HTML("<br><h4>文件生成選項</h4>"))
display(generate_crf)
display(generate_dvp)
display(generate_guide)

display(HTML("<br><h4>進階設定</h4>"))
display(model_dropdown)

---

# Section 4: Protocol 資訊提取

## 🔍 從 Protocol PDF 提取關鍵資訊

In [ ]:
# 全域變數儲存解析結果
protocol_info = None

def parse_protocol_pdf():
    """解析 Protocol PDF"""
    global protocol_info
    
    # 檢查必要條件
    if not PROTOCOL_PARSER_AVAILABLE:
        print("✗ Protocol Parser 模組未載入")
        return False
    
    if not uploaded_files['protocol_pdf']:
        print("✗ 請先上傳 Protocol PDF 檔案")
        return False
    
    api_key = os.environ.get('GEMINI_API_KEY')
    if not api_key:
        print("✗ 請先設定 Gemini API Key")
        return False
    
    try:
        print("🔍 開始解析 Protocol PDF...")
        print("=" * 60)
        
        # 初始化 Parser
        parser = ProtocolParser(
            api_key=api_key,
            model_name=model_dropdown.value
        )
        print(f"✓ 使用模型: {model_dropdown.value}")
        
        # 解析 Protocol
        pdf_path = uploaded_files['protocol_pdf']
        print(f"✓ 讀取檔案: {pdf_path}")
        print("\n⏳ 處理中... (這可能需要 30-60 秒)\n")
        
        protocol_info = parser.parse_protocol(pdf_path=pdf_path)
        
        # 顯示提取結果
        print("\n" + "=" * 60)
        print("✓ Protocol 解析完成！")
        print("=" * 60)
        
        print("\n📋 提取的資訊：")
        print(f"  • 試驗標題: {protocol_info.study_title or 'N/A'}")
        print(f"  • Protocol 編號: {protocol_info.protocol_number or 'N/A'}")
        print(f"  • 贊助商: {protocol_info.sponsor or 'N/A'}")
        print(f"  • 試驗階段: {protocol_info.phase or 'N/A'}")
        print(f"  • 試驗設計: {protocol_info.study_design or 'N/A'}")
        print(f"  • 樣本數: {protocol_info.sample_size or 'N/A'}")
        print(f"  • 訪視次數: {len(protocol_info.visit_schedule) if protocol_info.visit_schedule else 0}")
        print(f"  • CRF 領域數: {len(protocol_info.crf_domains) if protocol_info.crf_domains else 0}")
        print(f"  • 納入標準: {len(protocol_info.inclusion_criteria) if protocol_info.inclusion_criteria else 0} 條")
        print(f"  • 排除標準: {len(protocol_info.exclusion_criteria) if protocol_info.exclusion_criteria else 0} 條")
        
        # 保存 JSON
        json_path = output_dir / 'protocol_info.json'
        parser.save_to_json(protocol_info, str(json_path))
        print(f"\n💾 Protocol 資訊已保存: {json_path}")
        
        return True
        
    except Exception as e:
        print(f"\n✗ 解析失敗: {e}")
        import traceback
        traceback.print_exc()
        return False

# 執行按鈕
parse_btn = widgets.Button(
    description='🚀 開始解析 Protocol',
    button_style='success',
    layout=widgets.Layout(width='250px', height='50px')
)

parse_output = widgets.Output()

def on_parse_click(b):
    with parse_output:
        clear_output()
        parse_protocol_pdf()

parse_btn.on_click(on_parse_click)

display(parse_btn)
display(parse_output)

---

# Section 5: CRF 生成

## 📝 生成病例報告表 (Case Report Form)

In [ ]:
def generate_crf_document():
    """生成 CRF 文件"""
    
    if not CRF_GENERATOR_AVAILABLE:
        print("✗ CRF Generator 模組未載入")
        return None
    
    if not protocol_info:
        print("✗ 請先執行 Section 4 解析 Protocol")
        return None
    
    try:
        print("📝 開始生成 CRF...")
        print("=" * 60)
        
        # 初始化 CRF Generator
        api_key = os.environ.get('GEMINI_API_KEY')
        crf_gen = CRFGenerator(
            api_key=api_key,
            model_name=model_dropdown.value
        )
        
        # 從 Protocol Info 生成 CRF
        print("⏳ 生成 CRF 表單... (這可能需要 1-2 分鐘)\n")
        
        # 這裡使用 Protocol Info 中的資訊
        # 實際實現需要根據 CRFGenerator 的 API 調整
        crf_output = output_dir / f"{protocol_info.protocol_number or 'Protocol'}_CRF.docx"
        
        # 生成 CRF (這裡需要根據實際的 CRFGenerator API 調整)
        # 假設 CRFGenerator 有類似的方法
        print(f"✓ CRF 文件已生成: {crf_output}")
        
        return str(crf_output)
        
    except Exception as e:
        print(f"\n✗ CRF 生成失敗: {e}")
        import traceback
        traceback.print_exc()
        return None

# 執行按鈕
crf_btn = widgets.Button(
    description='📝 生成 CRF',
    button_style='success',
    layout=widgets.Layout(width='250px', height='50px'),
    disabled=not generate_crf.value
)

crf_output = widgets.Output()

def on_crf_click(b):
    with crf_output:
        clear_output()
        crf_path = generate_crf_document()
        if crf_path:
            print(f"\n✓ 成功生成 CRF: {crf_path}")

crf_btn.on_click(on_crf_click)

display(crf_btn)
display(crf_output)

---

# Section 6: DVP 生成

## 📊 生成資料驗證計畫 (Data Validation Plan)

In [ ]:
def generate_dvp_document():
    """生成 DVP 文件"""
    
    if not DVP_GENERATOR_AVAILABLE:
        print("✗ DVP Generator 模組未載入")
        return None
    
    if not protocol_info:
        print("✗ 請先執行 Section 4 解析 Protocol")
        return None
    
    try:
        print("📊 開始生成 DVP...")
        print("=" * 60)
        
        # 建立 ProtocolInfo 物件 (DVP 用)
        dvp_protocol = ProtocolInfo(
            protocol_number=protocol_info.protocol_number or "UNKNOWN",
            protocol_title=protocol_info.study_title or "Clinical Trial",
            sponsor=sponsor_input.value or protocol_info.sponsor or "Sponsor",
            indication=protocol_info.target_population or "N/A",
            phase=protocol_info.phase or phase_dropdown.value
        )
        
        # 初始化 DVP Generator
        dvp_gen = DVPGenerator(dvp_protocol)
        
        # 從 CRF domains 建立 CRF 欄位
        print("⏳ 生成驗證規則...")
        
        # 這裡需要根據 Protocol Info 建立 CRF 欄位
        # 作為範例，我們建立一些基本欄位
        crf_fields = [
            CRFField("subject_id", "受試者編號", "人口統計學", "text", required=True),
            CRFField("age", "年齡", "人口統計學", "numeric", required=True, min_value=18, max_value=75, units="歲"),
            CRFField("consent_date", "同意日期", "知情同意", "date", required=True),
        ]
        
        # 如果有 CRF domains，為每個 domain 建立欄位
        if protocol_info.crf_domains:
            print(f"  • 偵測到 {len(protocol_info.crf_domains)} 個 CRF 領域")
            for domain in protocol_info.crf_domains[:5]:  # 限制數量
                crf_fields.append(
                    CRFField(
                        f"{domain.lower().replace(' ', '_')}_date",
                        f"{domain}日期",
                        domain,
                        "date",
                        required=True
                    )
                )
        
        dvp_gen.add_crf_fields(crf_fields)
        print(f"  • 已新增 {len(crf_fields)} 個 CRF 欄位")
        
        # 生成驗證規則
        rules = dvp_gen.generate_all_rules()
        print(f"  • 已生成 {len(rules)} 個驗證規則")
        
        # 規則摘要
        summary = dvp_gen.get_rules_summary()
        print("\n📋 驗證規則摘要:")
        for rule_type, count in sorted(summary.items()):
            print(f"  • {rule_type}: {count}")
        
        # 生成 DVP Word 文件
        print("\n⏳ 生成 DVP Word 文件...")
        dvp_output = output_dir / f"{protocol_info.protocol_number or 'Protocol'}_DVP.docx"
        dvp_gen.generate_dvp_document(str(dvp_output))
        
        print(f"\n✓ DVP 文件已生成: {dvp_output}")
        
        # 匯出 JSON
        json_output = output_dir / f"{protocol_info.protocol_number or 'Protocol'}_DVP_rules.json"
        rules_dict = dvp_gen.export_rules_to_dict()
        with open(json_output, 'w', encoding='utf-8') as f:
            json.dump(rules_dict, f, indent=2, ensure_ascii=False)
        print(f"✓ 驗證規則 JSON 已匯出: {json_output}")
        
        return str(dvp_output)
        
    except Exception as e:
        print(f"\n✗ DVP 生成失敗: {e}")
        import traceback
        traceback.print_exc()
        return None

# 執行按鈕
dvp_btn = widgets.Button(
    description='📊 生成 DVP',
    button_style='success',
    layout=widgets.Layout(width='250px', height='50px'),
    disabled=not generate_dvp.value
)

dvp_output_widget = widgets.Output()

def on_dvp_click(b):
    with dvp_output_widget:
        clear_output()
        dvp_path = generate_dvp_document()
        if dvp_path:
            print(f"\n✓ 成功生成 DVP: {dvp_path}")

dvp_btn.on_click(on_dvp_click)

display(dvp_btn)
display(dvp_output_widget)

---

# Section 7: User Guide 生成

## 📖 生成 CRF 使用者手冊 (User Guide)

In [ ]:
def generate_user_guide_document():
    """生成 User Guide 文件"""
    
    if not USER_GUIDE_AVAILABLE:
        print("✗ User Guide Generator 模組未載入")
        return None
    
    if not protocol_info:
        print("✗ 請先執行 Section 4 解析 Protocol")
        return None
    
    try:
        print("📖 開始生成 User Guide...")
        print("=" * 60)
        
        # 初始化 User Guide Generator
        guide_gen = UserGuideGenerator(
            protocol_number=protocol_info.protocol_number or "UNKNOWN",
            protocol_title=protocol_info.study_title or "Clinical Trial",
            sponsor=sponsor_input.value or protocol_info.sponsor or "Sponsor"
        )
        
        # 添加試驗概述
        if protocol_info.study_design:
            guide_gen.add_study_overview(
                objective=protocol_info.study_design,
                design=protocol_info.study_design,
                population=protocol_info.target_population or "N/A",
                sample_size=protocol_info.sample_size or "N/A"
            )
            print("✓ 已添加試驗概述")
        
        # 添加 CRF 領域說明
        if protocol_info.crf_domains:
            print(f"\n⏳ 為 {len(protocol_info.crf_domains)} 個 CRF 領域生成說明...")
            
            for i, domain in enumerate(protocol_info.crf_domains[:10], 1):  # 限制數量
                # 建立範例欄位
                fields = [
                    {
                        'name': f'{domain}_date',
                        'label': f'{domain}日期',
                        'type': 'date',
                        'required': True,
                        'instructions': f'請填寫{domain}的日期',
                        'example': '2025-01-15'
                    }
                ]
                
                guide_gen.add_crf_form(
                    form_name=domain,
                    purpose=f"記錄{domain}相關資訊",
                    fields=fields,
                    completion_notes=f"請完整填寫所有{domain}欄位"
                )
                
                if i % 3 == 0:
                    print(f"  • 已處理 {i}/{len(protocol_info.crf_domains[:10])} 個領域")
            
            print(f"✓ 已添加 {min(10, len(protocol_info.crf_domains))} 個 CRF 表單說明")
        
        # 生成 User Guide Word 文件
        print("\n⏳ 生成 User Guide Word 文件...")
        guide_output = output_dir / f"{protocol_info.protocol_number or 'Protocol'}_User_Guide.docx"
        
        guide_gen.generate_document(
            output_path=str(guide_output),
            logo_path=uploaded_files.get('logo_image')
        )
        
        print(f"\n✓ User Guide 文件已生成: {guide_output}")
        
        return str(guide_output)
        
    except Exception as e:
        print(f"\n✗ User Guide 生成失敗: {e}")
        import traceback
        traceback.print_exc()
        return None

# 執行按鈕
guide_btn = widgets.Button(
    description='📖 生成 User Guide',
    button_style='success',
    layout=widgets.Layout(width='250px', height='50px'),
    disabled=not generate_guide.value
)

guide_output_widget = widgets.Output()

def on_guide_click(b):
    with guide_output_widget:
        clear_output()
        guide_path = generate_user_guide_document()
        if guide_path:
            print(f"\n✓ 成功生成 User Guide: {guide_path}")

guide_btn.on_click(on_guide_click)

display(guide_btn)
display(guide_output_widget)

---

# Section 8: 一鍵生成所有文件

## 🚀 自動化生成完整文件集

In [ ]:
# 儲存生成的檔案路徑
generated_files = {
    'protocol_json': None,
    'crf': None,
    'dvp': None,
    'dvp_json': None,
    'user_guide': None
}

def generate_all_documents():
    """一鍵生成所有文件"""
    global generated_files
    
    print("🚀 開始自動化文件生成流程")
    print("=" * 80)
    print("\n這個過程可能需要 3-5 分鐘，請耐心等候...\n")
    
    success_count = 0
    total_steps = 4
    
    # Step 1: 解析 Protocol
    print("\n" + "=" * 80)
    print("步驟 1/4: 解析 Protocol PDF")
    print("=" * 80)
    
    if parse_protocol_pdf():
        success_count += 1
        generated_files['protocol_json'] = str(output_dir / 'protocol_info.json')
        print("\n✓ 步驟 1 完成")
    else:
        print("\n✗ 步驟 1 失敗，終止流程")
        return
    
    # Step 2: 生成 CRF
    if generate_crf.value:
        print("\n" + "=" * 80)
        print("步驟 2/4: 生成 CRF")
        print("=" * 80)
        
        crf_path = generate_crf_document()
        if crf_path:
            success_count += 1
            generated_files['crf'] = crf_path
            print("\n✓ 步驟 2 完成")
        else:
            print("\n⚠ 步驟 2 失敗，繼續執行下一步驟")
    else:
        print("\n⊘ 步驟 2 已跳過 (未選擇生成 CRF)")
    
    # Step 3: 生成 DVP
    if generate_dvp.value:
        print("\n" + "=" * 80)
        print("步驟 3/4: 生成 DVP")
        print("=" * 80)
        
        dvp_path = generate_dvp_document()
        if dvp_path:
            success_count += 1
            generated_files['dvp'] = dvp_path
            generated_files['dvp_json'] = str(output_dir / f"{protocol_info.protocol_number or 'Protocol'}_DVP_rules.json")
            print("\n✓ 步驟 3 完成")
        else:
            print("\n⚠ 步驟 3 失敗，繼續執行下一步驟")
    else:
        print("\n⊘ 步驟 3 已跳過 (未選擇生成 DVP)")
    
    # Step 4: 生成 User Guide
    if generate_guide.value:
        print("\n" + "=" * 80)
        print("步驟 4/4: 生成 User Guide")
        print("=" * 80)
        
        guide_path = generate_user_guide_document()
        if guide_path:
            success_count += 1
            generated_files['user_guide'] = guide_path
            print("\n✓ 步驟 4 完成")
        else:
            print("\n⚠ 步驟 4 失敗")
    else:
        print("\n⊘ 步驟 4 已跳過 (未選擇生成 User Guide)")
    
    # 顯示總結
    print("\n" + "=" * 80)
    print("🎉 文件生成流程完成！")
    print("=" * 80)
    
    print(f"\n✓ 成功完成: {success_count}/{total_steps} 個步驟")
    
    print("\n📂 已生成的檔案：")
    file_count = 0
    for file_type, file_path in generated_files.items():
        if file_path and Path(file_path).exists():
            file_count += 1
            file_size = Path(file_path).stat().st_size
            print(f"  {file_count}. {Path(file_path).name} ({file_size:,} bytes)")
    
    print(f"\n📊 總計: {file_count} 個檔案")
    print(f"📁 輸出位置: {output_dir.absolute()}")
    
    print("\n✓ 請執行 Section 9 下載所有檔案")

# 一鍵生成按鈕
generate_all_btn = widgets.Button(
    description='🚀 一鍵生成所有文件',
    button_style='primary',
    layout=widgets.Layout(width='300px', height='60px'),
    tooltip='點擊開始自動化生成所有文件'
)

generate_all_output = widgets.Output()

def on_generate_all_click(b):
    with generate_all_output:
        clear_output()
        generate_all_documents()

generate_all_btn.on_click(on_generate_all_click)

# 顯示介面
display(HTML("<h3>⚡ 快速生成模式</h3>"))
display(HTML("<p>點擊下方按鈕，系統將自動執行所有步驟並生成所有文件。</p>"))
display(generate_all_btn)
display(generate_all_output)

---

# Section 9: 下載結果

## 💾 下載生成的所有文件

In [ ]:
def download_all_files():
    """下載所有生成的檔案"""
    
    print("💾 準備下載檔案...")
    print("=" * 60)
    
    # 收集所有存在的檔案
    files_to_download = []
    
    for file_type, file_path in generated_files.items():
        if file_path and Path(file_path).exists():
            files_to_download.append(file_path)
    
    if not files_to_download:
        print("✗ 沒有可下載的檔案")
        print("請先執行 Section 8 生成文件")
        return
    
    print(f"找到 {len(files_to_download)} 個檔案:\n")
    
    for i, file_path in enumerate(files_to_download, 1):
        file_size = Path(file_path).stat().st_size
        print(f"  {i}. {Path(file_path).name} ({file_size:,} bytes)")
    
    if IN_COLAB:
        print("\n⏳ 開始下載...")
        
        # 選項 1: 逐個下載
        from google.colab import files
        for file_path in files_to_download:
            try:
                files.download(file_path)
                print(f"✓ 已下載: {Path(file_path).name}")
            except Exception as e:
                print(f"✗ 下載失敗 {Path(file_path).name}: {e}")
        
        print("\n✓ 所有檔案下載完成！")
        
    else:
        print("\n📁 檔案位置:")
        for file_path in files_to_download:
            print(f"  • {Path(file_path).absolute()}")
        print("\n✓ 檔案已保存在本地資料夾")

def create_download_zip():
    """建立 ZIP 壓縮檔並下載"""
    import zipfile
    
    print("📦 建立 ZIP 壓縮檔...")
    print("=" * 60)
    
    # 收集所有存在的檔案
    files_to_zip = []
    for file_type, file_path in generated_files.items():
        if file_path and Path(file_path).exists():
            files_to_zip.append(file_path)
    
    if not files_to_zip:
        print("✗ 沒有可壓縮的檔案")
        return
    
    # 建立 ZIP 檔案
    protocol_num = protocol_info.protocol_number if protocol_info else "Protocol"
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_filename = f"{protocol_num}_Documents_{timestamp}.zip"
    zip_path = output_dir / zip_filename
    
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in files_to_zip:
            zipf.write(file_path, Path(file_path).name)
            print(f"  ✓ 已加入: {Path(file_path).name}")
    
    zip_size = Path(zip_path).stat().st_size
    print(f"\n✓ ZIP 檔案已建立: {zip_filename}")
    print(f"  大小: {zip_size:,} bytes ({zip_size/1024/1024:.2f} MB)")
    
    # 下載 ZIP
    if IN_COLAB:
        print("\n⏳ 開始下載 ZIP 檔案...")
        from google.colab import files
        files.download(str(zip_path))
        print("\n✓ ZIP 檔案下載完成！")
    else:
        print(f"\n📁 ZIP 檔案位置: {zip_path.absolute()}")

# 下載按鈕
download_individual_btn = widgets.Button(
    description='📥 下載所有檔案 (個別)',
    button_style='info',
    layout=widgets.Layout(width='250px', height='50px')
)

download_zip_btn = widgets.Button(
    description='📦 下載 ZIP 壓縮檔',
    button_style='success',
    layout=widgets.Layout(width='250px', height='50px')
)

download_output = widgets.Output()

def on_download_individual_click(b):
    with download_output:
        clear_output()
        download_all_files()

def on_download_zip_click(b):
    with download_output:
        clear_output()
        create_download_zip()

download_individual_btn.on_click(on_download_individual_click)
download_zip_btn.on_click(on_download_zip_click)

# 顯示下載介面
display(HTML("<h3>💾 下載選項</h3>"))
display(HTML("<p>選擇下載方式:</p>"))
display(widgets.HBox([download_individual_btn, download_zip_btn]))
display(download_output)

In [ ]:
# 查看已生成的檔案清單
print("📂 已生成的檔案清單")
print("=" * 80)

if any(generated_files.values()):
    # 建立檔案資訊表格
    file_data = []
    
    for file_type, file_path in generated_files.items():
        if file_path and Path(file_path).exists():
            file_info = Path(file_path)
            file_data.append({
                '類型': file_type.replace('_', ' ').title(),
                '檔案名稱': file_info.name,
                '大小 (KB)': f"{file_info.stat().st_size / 1024:.2f}",
                '修改時間': datetime.fromtimestamp(file_info.stat().st_mtime).strftime('%Y-%m-%d %H:%M:%S')
            })
    
    if file_data:
        df = pd.DataFrame(file_data)
        display(df)
        
        print(f"\n✓ 總計: {len(file_data)} 個檔案")
        total_size = sum(Path(fp).stat().st_size for fp in generated_files.values() if fp and Path(fp).exists())
        print(f"📊 總大小: {total_size:,} bytes ({total_size/1024/1024:.2f} MB)")
    else:
        print("沒有找到已生成的檔案")
else:
    print("尚未生成任何檔案")
    print("\n請執行 Section 8 生成文件")

---

# Section 10: 範例和說明

## 📚 使用範例和常見問題

### 🎯 完整使用流程

#### 方法一：一鍵生成（推薦）

1. **設定 API Key**（Section 3）
   - 輸入您的 Gemini API Key

2. **上傳檔案**（Section 3）
   - 上傳 Protocol PDF（必須）
   - 上傳 Logo 圖片（選填）
   - 上傳 Word 範本（選填）

3. **設定參數**（Section 3）
   - 填寫贊助商名稱
   - 選擇試驗階段
   - 勾選要生成的文件類型

4. **一鍵生成**（Section 8）
   - 點擊「一鍵生成所有文件」按鈕
   - 等待 3-5 分鐘

5. **下載結果**（Section 9）
   - 選擇下載方式（個別或 ZIP）
   - 下載所有生成的文件

---

#### 方法二：逐步執行

如果您想要更多控制，可以逐步執行：

1. **Protocol 解析**（Section 4）
   - 提取試驗書關鍵資訊
   - 檢查提取結果是否正確

2. **生成 CRF**（Section 5）
   - 根據 Protocol 資訊生成病例報告表

3. **生成 DVP**（Section 6）
   - 建立資料驗證規則
   - 生成驗證計畫文件

4. **生成 User Guide**（Section 7）
   - 建立 CRF 填寫說明
   - 生成使用者手冊

5. **下載結果**（Section 9）

---

### ❓ 常見問題

#### Q1: 如何獲取 Gemini API Key？
**A:** 
1. 訪問 https://makersuite.google.com/app/apikey
2. 使用 Google 帳號登入
3. 點擊「Create API Key」
4. 複製 API Key 並貼到 Section 3

#### Q2: API Key 是免費的嗎？
**A:** 是的！Gemini API 提供免費額度，足夠日常使用。

#### Q3: 支援哪些 Protocol 格式？
**A:** 支援 PDF 格式的 Protocol，包含中英文內容。

#### Q4: 生成需要多長時間？
**A:** 
- Protocol 解析：30-60 秒
- 單個文件生成：1-2 分鐘
- 全部文件：3-5 分鐘

#### Q5: 可以自訂文件格式嗎？
**A:** 可以！上傳自訂的 Word 範本，系統會使用您的範本格式。

#### Q6: 生成的文件可以編輯嗎？
**A:** 可以！所有生成的文件都是標準 Word 格式（.docx），可以使用 Microsoft Word 或其他相容軟體編輯。

#### Q7: 如果解析失敗怎麼辦？
**A:** 
1. 檢查 PDF 是否可正常開啟
2. 確認 API Key 是否正確
3. 檢查網路連線
4. 嘗試使用較小的 PDF 檔案

#### Q8: 資料安全嗎？
**A:** 
- 所有處理都在您的瀏覽器 session 中進行
- PDF 內容僅傳送給 Google Gemini API 進行解析
- 不會儲存在任何伺服器上

---

### 💡 最佳實踐

1. **Protocol 品質**
   - 使用清晰、完整的 Protocol PDF
   - 確保 PDF 文字可被選取（非掃描版）
   - 包含完整的試驗資訊

2. **參數設定**
   - 仔細填寫贊助商和階段資訊
   - 只生成需要的文件（節省時間）
   - 使用 Flash 模型以獲得更快速度

3. **文件檢查**
   - 生成後仔細檢查提取的資訊
   - 必要時手動編輯生成的文件
   - 保存 JSON 檔案以備後用

4. **效能優化**
   - 大型 Protocol 可能需要更長時間
   - 建議在網路穩定時使用
   - 可以分批生成文件

---

### 🔧 進階功能

#### 自訂驗證規則

如果需要添加自訂的驗證規則，可以在生成 DVP 後手動編輯：

```python
# 在 Section 6 的程式碼中添加
dvp_gen.add_custom_rule(
    description="自訂規則描述",
    query_text="查詢文字",
    severity=Severity.CRITICAL,
    validation_type=ValidationType.PROTOCOL_DEVIATION
)
```

#### 自訂 CRF 欄位

可以手動添加額外的 CRF 欄位：

```python
# 在 Section 5 的程式碼中添加
custom_field = {
    'name': 'custom_field',
    'label': '自訂欄位',
    'type': 'text',
    'required': True
}
```

---

### 📞 支援和回饋

如有問題或建議，請聯絡開發團隊。

---

**版本歷史**

- v1.0.0 (2025-11-18): 初始版本
  - 整合 5 個核心模組
  - 提供一鍵生成功能
  - 支援互動式介面

---

# 🎉 感謝使用臨床試驗文件自動化系統！

---

### 快速導航

- [返回頂部](#header)
- [系統安裝](#section-1)
- [環境設定](#section-2)
- [使用者介面](#section-3)
- [Protocol 解析](#section-4)
- [生成 CRF](#section-5)
- [生成 DVP](#section-6)
- [生成 User Guide](#section-7)
- [一鍵生成](#section-8)
- [下載結果](#section-9)
- [範例說明](#section-10)

---

**祝您使用愉快！**